In [1]:
# ============================================================
# DAILY — Single Prophet Univariate Fit + Forecast to FY end
# Train: 2021-04-01 .. 2025-12-31
# Forecast: 2026-01-01 .. 2027-03-31
# Outputs:
#   - history_fit_daily_org.csv
#   - forecast_daily_to_fy2026_27_org.csv
#   - forecast_fy_totals_org.csv
# ============================================================

import os
import warnings
import numpy as np
import pandas as pd

# -----------------------------
# Config
# -----------------------------
INPUT_PATH = r"\\Sherwood\accounting\Share\Finance Meetings & Reports\Monthly Reports\BCM Report Monthly\Power BI Report\Forecasting Method\0_data_input_monthly_DAILY.csv"  # <- update if needed
OUTPUT_DIR = r"\\Sherwood\accounting\Share\Finance Meetings & Reports\Monthly Reports\BCM Report Monthly\Power BI Report\Forecasting Method\Output_Daily"  # <- update if needed
os.makedirs(OUTPUT_DIR, exist_ok=True)

DATE_COL = "ACC_DAY"
Y_COL = "TOT_PAID"
BUD_COL = "BUDGET_AMT"
CASE_COL = "NBR_CASE"

TRAIN_START = pd.Timestamp("2021-04-01")
TRAIN_END   = pd.Timestamp("2025-12-31")

FORECAST_START = pd.Timestamp("2026-01-01")
FORECAST_END   = pd.Timestamp("2027-03-31")  # FY2026-27 ends Mar 31, 2027

# Prophet hyperparameters (same pattern as your prior code)
CP = 0.05
SP = 10.0
N_CHANGEPOINTS = 25
CHANGEPOINT_RANGE = 0.95

OUT_HISTORY = os.path.join(OUTPUT_DIR, "history_fit_daily_org.csv")
OUT_FORECAST = os.path.join(OUTPUT_DIR, "forecast_daily_to_fy2026_27_org.csv")
OUT_FY_TOTALS = os.path.join(OUTPUT_DIR, "forecast_fy_totals_org.csv")

# -----------------------------
# Prophet import
# -----------------------------
try:
    from prophet import Prophet
except Exception:
    raise RuntimeError("Prophet not available. Install with: pip install prophet")

# -----------------------------
# Helpers
# -----------------------------
def ensure_dt(s):
    return pd.to_datetime(s, errors="coerce")

def safe_to_csv(df_: pd.DataFrame, path: str, **kwargs) -> str:
    parent = os.path.dirname(path)
    if parent:
        os.makedirs(parent, exist_ok=True)
    df_.to_csv(path, **kwargs)
    return path

def fy_start_year(dt: pd.Timestamp) -> int:
    dt = pd.Timestamp(dt)
    return dt.year if dt.month >= 4 else dt.year - 1

def fiscal_year_label(dt: pd.Timestamp) -> str:
    y = fy_start_year(pd.Timestamp(dt))
    return f"FY{y}-{str(y+1)[-2:]}"

def prophet_fit_predict(train_series: pd.Series, predict_index: pd.DatetimeIndex) -> pd.Series:
    """
    Fits Prophet on train_series (daily) and predicts on predict_index.
    Returns yhat as pd.Series indexed by predict_index.
    """
    s = train_series.astype(float).dropna()
    if s.empty:
        return pd.Series(np.nan, index=predict_index, dtype=float)

    train_df = s.reset_index()
    train_df.columns = ["ds", "y"]

    m = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=False,
        daily_seasonality=False,
        seasonality_mode="additive",
        changepoint_prior_scale=float(CP),
        seasonality_prior_scale=float(SP),
        n_changepoints=int(N_CHANGEPOINTS),
        changepoint_range=float(CHANGEPOINT_RANGE),
    )
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        m.fit(train_df)

    fut = pd.DataFrame({"ds": predict_index})
    yhat = m.predict(fut)["yhat"].values
    return pd.Series(yhat, index=predict_index, dtype=float)

# -----------------------------
# Read & prep daily data
# -----------------------------
df = pd.read_csv(INPUT_PATH)
df.columns = [c.strip() for c in df.columns]

required = [DATE_COL, Y_COL, BUD_COL, CASE_COL]
missing = [c for c in required if c not in df.columns]
if missing:
    raise ValueError(f"Missing required columns: {missing}")

df[DATE_COL] = ensure_dt(df[DATE_COL])
df = df.dropna(subset=[DATE_COL]).copy()

for c in [Y_COL, BUD_COL, CASE_COL]:
    df[c] = pd.to_numeric(df[c], errors="coerce")

df = df.sort_values(DATE_COL).set_index(DATE_COL).asfreq("D")

# -----------------------------
# Train slice (exact date window you requested)
# -----------------------------
train_df = df.loc[TRAIN_START:TRAIN_END].copy()
train_y = train_df[Y_COL].dropna()

if train_y.empty:
    raise ValueError("Training slice has no non-null TOT_PAID values. Check dates / data.")

# -----------------------------
# In-sample fitted values (history)
# Predict over the training index so you get yhat_org for 2021-04-01..2025-12-31
# -----------------------------
train_index = pd.date_range(TRAIN_START, TRAIN_END, freq="D")
yhat_hist = prophet_fit_predict(train_y, train_index)

hist_out = df.loc[TRAIN_START:TRAIN_END, [Y_COL, BUD_COL, CASE_COL]].copy()
hist_out = hist_out.reindex(train_index)
hist_out.index.name = "ds"
hist_out = hist_out.reset_index().rename(columns={
    Y_COL: "y_actual",
    BUD_COL: "Budget_AMT",
    CASE_COL: "Nbr_Cases",
})
hist_out["model"] = "M2_Prophet_univariate_daily"
hist_out["yhat_org"] = yhat_hist.values
hist_out["FY"] = hist_out["ds"].apply(fiscal_year_label)

safe_to_csv(hist_out, OUT_HISTORY, index=False)

# -----------------------------
# Forecast future to FY end (2027-03-31)
# -----------------------------
future_index = pd.date_range(FORECAST_START, FORECAST_END, freq="D")
yhat_future = prophet_fit_predict(train_y, future_index)

fc_out = pd.DataFrame({
    "ds": future_index,
    "model": "M2_Prophet_univariate_daily",
    "yhat_org": yhat_future.values,
})
fc_out["FY"] = fc_out["ds"].apply(fiscal_year_label)

safe_to_csv(fc_out, OUT_FORECAST, index=False)

# FY totals from forecast
fy_totals = (
    fc_out.groupby("FY", as_index=False)
          .agg(yhat_org_total=("yhat_org", "sum"))
)
safe_to_csv(fy_totals, OUT_FY_TOTALS, index=False)

print("✅ Done.")
print("History (in-sample fit):", OUT_HISTORY)
print("Forecast (future):      ", OUT_FORECAST)
print("FY totals:              ", OUT_FY_TOTALS)

20:56:46 - cmdstanpy - INFO - Chain [1] start processing
20:56:47 - cmdstanpy - INFO - Chain [1] done processing
20:56:49 - cmdstanpy - INFO - Chain [1] start processing
20:56:49 - cmdstanpy - INFO - Chain [1] done processing


✅ Done.
History (in-sample fit): \\Sherwood\accounting\Share\Finance Meetings & Reports\Monthly Reports\BCM Report Monthly\Power BI Report\Forecasting Method\Output_Daily\history_fit_daily_org.csv
Forecast (future):       \\Sherwood\accounting\Share\Finance Meetings & Reports\Monthly Reports\BCM Report Monthly\Power BI Report\Forecasting Method\Output_Daily\forecast_daily_to_fy2026_27_org.csv
FY totals:               \\Sherwood\accounting\Share\Finance Meetings & Reports\Monthly Reports\BCM Report Monthly\Power BI Report\Forecasting Method\Output_Daily\forecast_fy_totals_org.csv


In [4]:
# ============================================================
# DAILY — Single Prophet Univariate Fit
# Train: 2021-04-01 .. 2024-03-31 (end of FY2023-24)
# Test : 2025-04-01 .. 2025-12-31 (9 months in FY2025-26)
# Forecast: 2024-04-01 .. 2027-03-31 (end of FY2026-27)
#
# Outputs (in \\...\Forecasting Method\Output_Daily):
#   - daily_history_fit_train_only.csv
#   - daily_forecast_to_FY2026_27.csv
#   - daily_test_window_eval.csv
#   - daily_test_metrics_summary.csv
#   - forecast_fy_totals_org.csv
# ============================================================

import os
import warnings
import numpy as np
import pandas as pd

# -----------------------------
# Config (UNC paths)
# -----------------------------
BASE_DIR = r"\\Sherwood\accounting\Share\Finance Meetings & Reports\Monthly Reports\BCM Report Monthly\Power BI Report\Forecasting Method"
INPUT_PATH = os.path.join(BASE_DIR, "0_data_input_monthly_DAILY.csv")

OUTPUT_DIR = os.path.join(BASE_DIR, "Output_Daily")
os.makedirs(OUTPUT_DIR, exist_ok=True)

DATE_COL = "ACC_DAY"
Y_COL    = "TOT_PAID"
BUD_COL  = "BUDGET_AMT"
CASE_COL = "NBR_CASE"

TRAIN_START = pd.Timestamp("2021-04-01")
TRAIN_END   = pd.Timestamp("2024-03-31")

TEST_START  = pd.Timestamp("2025-04-01")
TEST_END    = pd.Timestamp("2025-12-31")

FORECAST_START = pd.Timestamp("2024-04-01")
FORECAST_END   = pd.Timestamp("2027-03-31")

# Prophet hyperparameters
CP = 0.05
SP = 10.0
N_CHANGEPOINTS = 25
CHANGEPOINT_RANGE = 0.95

OUT_HISTORY_TRAIN = os.path.join(OUTPUT_DIR, "daily_history_fit_train_only.csv")
OUT_FORECAST      = os.path.join(OUTPUT_DIR, "daily_forecast_to_FY2026_27.csv")
OUT_TEST_EVAL     = os.path.join(OUTPUT_DIR, "daily_test_window_eval.csv")
OUT_TEST_METRICS  = os.path.join(OUTPUT_DIR, "daily_test_metrics_summary.csv")
OUT_FY_TOTALS     = os.path.join(OUTPUT_DIR, "forecast_fy_totals_org.csv")

# -----------------------------
# Prophet import
# -----------------------------
try:
    from prophet import Prophet
except Exception:
    raise RuntimeError("Prophet not available. Install with: pip install prophet")

# -----------------------------
# Helpers
# -----------------------------
def ensure_dt(s):
    return pd.to_datetime(s, errors="coerce")

def safe_to_csv(df_: pd.DataFrame, path: str, **kwargs) -> str:
    parent = os.path.dirname(path)
    if parent:
        os.makedirs(parent, exist_ok=True)
    df_.to_csv(path, **kwargs)
    return path

def fy_start_year(dt: pd.Timestamp) -> int:
    dt = pd.Timestamp(dt)
    return dt.year if dt.month >= 4 else dt.year - 1

def fiscal_year_label(dt: pd.Timestamp) -> str:
    y = fy_start_year(pd.Timestamp(dt))
    return f"FY{y}-{str(y+1)[-2:]}"

def wape(y_true, y_pred):
    """WAPE (%): 100 * sum(|pred-actual|) / sum(|actual|)"""
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    m = np.isfinite(y_true) & np.isfinite(y_pred)
    if m.sum() == 0:
        return np.nan
    denom = np.nansum(np.abs(y_true[m]))
    if denom == 0 or np.isnan(denom):
        return np.nan
    return float(100.0 * (np.nansum(np.abs(y_pred[m] - y_true[m])) / denom))

def bias_wape(y_true, y_pred):
    """Bias% (WAPE-style): 100 * sum(pred-actual) / sum(|actual|)"""
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    m = np.isfinite(y_true) & np.isfinite(y_pred)
    if m.sum() == 0:
        return np.nan
    denom = np.nansum(np.abs(y_true[m]))
    if denom == 0 or np.isnan(denom):
        return np.nan
    return float(100.0 * (np.nansum(y_pred[m] - y_true[m]) / denom))

def coalesce_duplicate_columns(df: pd.DataFrame, colname: str) -> pd.DataFrame:
    """
    If df has duplicate columns with the exact same name (colname),
    collapse them into a single Series using first non-null across columns.
    """
    matches = [c for c in df.columns if c == colname]
    if len(matches) <= 1:
        return df

    # bfill across duplicate cols -> take first non-null per row
    combined = df[matches].bfill(axis=1).iloc[:, 0]
    df = df.drop(columns=matches)
    df[colname] = combined
    return df

def prophet_fit(train_series: pd.Series):
    s = train_series.astype(float).dropna()
    if s.empty:
        raise ValueError("Training series empty after dropping NA.")

    train_df = s.reset_index()
    train_df.columns = ["ds", "y"]

    m = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=False,
        daily_seasonality=False,
        seasonality_mode="additive",
        changepoint_prior_scale=float(CP),
        seasonality_prior_scale=float(SP),
        n_changepoints=int(N_CHANGEPOINTS),
        changepoint_range=float(CHANGEPOINT_RANGE),
    )
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        m.fit(train_df)
    return m

def prophet_predict(model, predict_index: pd.DatetimeIndex) -> pd.Series:
    fut = pd.DataFrame({"ds": predict_index})
    yhat = model.predict(fut)["yhat"].values
    return pd.Series(yhat, index=predict_index, dtype=float)

# -----------------------------
# Read & prep daily data
# -----------------------------
df = pd.read_csv(INPUT_PATH)
df.columns = [str(c).strip() for c in df.columns]

# Fix exact duplicate columns (common cause of pd.to_numeric TypeError)
for col in [DATE_COL, Y_COL, BUD_COL, CASE_COL]:
    df = coalesce_duplicate_columns(df, col)

required = [DATE_COL, Y_COL, BUD_COL, CASE_COL]
missing = [c for c in required if c not in df.columns]
if missing:
    raise ValueError(f"Missing required columns: {missing}")

df[DATE_COL] = ensure_dt(df[DATE_COL])
df = df.dropna(subset=[DATE_COL]).copy()

for c in [Y_COL, BUD_COL, CASE_COL]:
    df[c] = pd.to_numeric(df[c], errors="coerce")

df = df.sort_values(DATE_COL).set_index(DATE_COL).asfreq("D")

# -----------------------------
# Train and fit once
# -----------------------------
train_slice = df.loc[TRAIN_START:TRAIN_END].copy()
train_y = train_slice[Y_COL].dropna()
if train_y.empty:
    raise ValueError("Training slice has no non-null TOT_PAID values. Check dates/data.")

print(
    "Train window:", TRAIN_START.date(), "to", TRAIN_END.date(),
    "| y non-null:", int(train_y.notna().sum())
)

m = prophet_fit(train_y)

# -----------------------------
# In-sample fitted values over TRAIN window
# -----------------------------
train_index = pd.date_range(TRAIN_START, TRAIN_END, freq="D")
yhat_train = prophet_predict(m, train_index)

hist_out = df.loc[TRAIN_START:TRAIN_END, [Y_COL, BUD_COL, CASE_COL]].reindex(train_index).copy()
hist_out.index.name = "ds"
hist_out = hist_out.reset_index().rename(columns={
    Y_COL: "y_actual",
    BUD_COL: "Budget_AMT",
    CASE_COL: "Nbr_Cases",
})
hist_out["model"] = "M2_Prophet_univariate_daily_singlefit"
hist_out["yhat_org"] = yhat_train.values
hist_out["FY"] = hist_out["ds"].apply(fiscal_year_label)
safe_to_csv(hist_out, OUT_HISTORY_TRAIN, index=False)

# -----------------------------
# Forecast 2024-04-01 .. 2027-03-31
# -----------------------------
forecast_index = pd.date_range(FORECAST_START, FORECAST_END, freq="D")
yhat_fc = prophet_predict(m, forecast_index)

fc_out = pd.DataFrame({
    "ds": forecast_index,
    "model": "M2_Prophet_univariate_daily_singlefit",
    "yhat_org": yhat_fc.values,
})
fc_out["FY"] = fc_out["ds"].apply(fiscal_year_label)
safe_to_csv(fc_out, OUT_FORECAST, index=False)

# -----------------------------
# Test evaluation (Apr 1 2025..Dec 31 2025)
# -----------------------------
actual_test = df.loc[TEST_START:TEST_END, [Y_COL, BUD_COL, CASE_COL]].copy()
actual_test = actual_test.rename(columns={
    Y_COL: "y_actual",
    BUD_COL: "Budget_AMT",
    CASE_COL: "Nbr_Cases",
})
pred_test = fc_out.set_index("ds").loc[TEST_START:TEST_END, ["yhat_org"]].copy()

# ---- FIX: make ds robust after reset_index() (index might be named ACC_DAY, not "index") ----
eval_df = actual_test.join(pred_test, how="inner").reset_index()

if "ds" not in eval_df.columns:
    if DATE_COL in eval_df.columns:
        eval_df = eval_df.rename(columns={DATE_COL: "ds"})
    else:
        eval_df = eval_df.rename(columns={eval_df.columns[0]: "ds"})

eval_df["ds"] = ensure_dt(eval_df["ds"])
eval_df["FY"] = eval_df["ds"].apply(fiscal_year_label)
# -------------------------------------------------------------------------------------------

eval_df = eval_df[
    np.isfinite(eval_df["y_actual"]) & np.isfinite(eval_df["yhat_org"])
].copy()

safe_to_csv(eval_df, OUT_TEST_EVAL, index=False)

test_wape = wape(eval_df["y_actual"].values, eval_df["yhat_org"].values)
test_bias = bias_wape(eval_df["y_actual"].values, eval_df["yhat_org"].values)

test_metrics = pd.DataFrame([{
    "train_start": TRAIN_START.date(),
    "train_end": TRAIN_END.date(),
    "test_start": TEST_START.date(),
    "test_end": TEST_END.date(),
    "test_days_available": int(eval_df.shape[0]),
    "WAPE_pct_test_9m": test_wape,
    "Bias_WAPE_pct_test_9m": test_bias,
}])
safe_to_csv(test_metrics, OUT_TEST_METRICS, index=False)

# -----------------------------
# FY totals from forecast
# -----------------------------
fy_totals = (
    fc_out.groupby("FY", as_index=False)
         .agg(yhat_org_total=("yhat_org", "sum"))
)
safe_to_csv(fy_totals, OUT_FY_TOTALS, index=False)

print("\n✅ Done.")
print("Train history fit:", OUT_HISTORY_TRAIN)
print("Forecast file:    ", OUT_FORECAST)
print("Test eval file:   ", OUT_TEST_EVAL)
print("Test metrics:     ", OUT_TEST_METRICS)
print("FY totals:        ", OUT_FY_TOTALS)
print("\nTest window WAPE%:", test_wape)
print("Test window Bias%:", test_bias)

# Optional quick data sanity check for late 2025 actual availability
print("\nSanity check TOT_PAID (Oct-Dec 2025):")
print("Not NA:", int(df.loc["2025-10-01":"2025-12-31", Y_COL].notna().sum()))
print("Is  NA:", int(df.loc["2025-10-01":"2025-12-31", Y_COL].isna().sum()))

Train window: 2021-04-01 to 2024-03-31 | y non-null: 842


21:06:57 - cmdstanpy - INFO - Chain [1] start processing
21:06:57 - cmdstanpy - INFO - Chain [1] done processing



✅ Done.
Train history fit: \\Sherwood\accounting\Share\Finance Meetings & Reports\Monthly Reports\BCM Report Monthly\Power BI Report\Forecasting Method\Output_Daily\daily_history_fit_train_only.csv
Forecast file:     \\Sherwood\accounting\Share\Finance Meetings & Reports\Monthly Reports\BCM Report Monthly\Power BI Report\Forecasting Method\Output_Daily\daily_forecast_to_FY2026_27.csv
Test eval file:    \\Sherwood\accounting\Share\Finance Meetings & Reports\Monthly Reports\BCM Report Monthly\Power BI Report\Forecasting Method\Output_Daily\daily_test_window_eval.csv
Test metrics:      \\Sherwood\accounting\Share\Finance Meetings & Reports\Monthly Reports\BCM Report Monthly\Power BI Report\Forecasting Method\Output_Daily\daily_test_metrics_summary.csv
FY totals:         \\Sherwood\accounting\Share\Finance Meetings & Reports\Monthly Reports\BCM Report Monthly\Power BI Report\Forecasting Method\Output_Daily\forecast_fy_totals_org.csv

Test window WAPE%: 70.92864590696455
Test window Bias%: